In [1]:
import dotenv
dotenv.load_dotenv()
import os
import wandb

In [2]:
wandb.login(key = os.getenv("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/kilnaar/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [3]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, TrainingArguments
)
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
import torch

/home/kilnaar/anaconda3/envs/gemma-tune/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#wandb.init(project = "gemma-tune")

In [5]:
model_id = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast = True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit = True,
    device_map = "auto",
    torch_dtype = torch.bfloat16
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.77it/s]
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [6]:
ds = load_dataset("Abirate/english_quotes", split = "train[:2%]")

In [7]:
def format_fn(ex):
    return f"Quote: {ex['quote']}\nAuthor: {ex['author']}"

lora_cfg = LoraConfig(
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type = TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_cfg)

# --- Training -----------------------------------------------------
args = TrainingArguments(
    output_dir = "test-gemma-lora-adapter",
    per_device_train_batch_size = 4,
    num_train_epochs = 1,
    gradient_accumulation_steps = 4,
    learning_rate = 2e-4,
    fp16 = True,
    logging_steps = 10,
    save_steps = 100,
    save_total_limit = 2,

    # Disable W&B for now
    report_to=[]
)

trainer = SFTTrainer(
    model = model,
    args = args,
    train_dataset = ds,
    formatting_func = format_fn,
    processing_class = tokenizer,
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [8]:
trainer.train()

/home/kilnaar/anaconda3/envs/gemma-tune/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/kilnaar/anaconda3/envs/gemma-tune/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


TrainOutput(global_step=4, training_loss=1.604592204093933, metrics={'train_runtime': 3.3604, 'train_samples_per_second': 14.879, 'train_steps_per_second': 1.19, 'total_flos': 52251385405440.0, 'train_loss': 1.604592204093933})

In [ ]:
# model.save_pretrained("test-gemma-lora-adapter")
# tokenizer.save_pretrained("test-gemma-lora-adapter")